In [1]:
import pandas as pd
import numpy as np
import re

In [7]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
import torch

In [3]:
df = pd.read_csv("data/anonymized_results.csv")

In [4]:
for col in df.columns:
    if list(np.unique(df[col].astype(str))) == ["-"]:
        df.drop(columns=[col], inplace=True)

print(len(df))

282


In [8]:
MODEL = './data/feedbackBert'
TOKZER = "google/roberta2roberta_L-24_bbc"
#'sshleifer/distilbart-cnn-12-6'
#'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
#'sentence-transformers/bert-base-nli-mean-tokens' #"bert-base-uncased" #f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(TOKZER)
model = AutoModelForSeq2SeqLM.from_pretrained(TOKZER)
#AutoModel.from_pretrained(TOKZER)
new_model = model.base_model

Downloading:   0%|          | 0.00/1.70G [00:00<?, ?B/s]

In [9]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


In [16]:
# def get_768_features(text):
#     if len(text) > 500:
#         text = text[-500:]
#     encoded_input = tokenizer([text], padding=True, truncation=True, return_tensors='pt')
#     with torch.no_grad():
#         model_output = new_model(**encoded_input)
#     sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
#     return sentence_embeddings

def get_768_features(text):
    if len(text) > 500:
        text = text[-500:]
    encoded_input = tokenizer([text], padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = new_model(**encoded_input)
    scores = model_output[0][0].detach().numpy()
    return scores[0]

### Feedback embedings

In [11]:
"""
Give a **specific
the easiest
"""
question_one_columns = [col for col in df.columns if "Give a **specific" in col]
question_two_columns = [col for col in df.columns if "the easiest" in col]

In [14]:
for col in question_one_columns:
    week_num = col[4]
    df[col].fillna("None", inplace=True)
    df.loc[df[col] == "", col] = "none"
    df[f"week{week_num}_q1_embeding"] = df[col].apply(get_768_features)
for col in question_two_columns:
    week_num = col[4]
    df[col].fillna("None", inplace=True)
    df.loc[df[col] == "", col] = "none"
    df[f"week{week_num}_q2_embeding"] = df[col].apply(get_768_features)


ValueError: You have to specify either input_ids or inputs_embeds

### topics embedings

In [201]:
with open("data/topics_list.txt", "r") as f:
    topics = f.readlines()
with open("data/labs_list.txt", "r") as f:
    labs = f.read().split("\n")

In [202]:
topics = [re.sub("(\d+.\d+ )", "", top).rstrip("\n") for top in topics]


In [203]:
#topics = topics + labs

In [204]:
topics

['Programming',
 'input and output',
 'whitespace',
 'Variables',
 'assignments',
 'Objects',
 'Float',
 'Arithmetic',
 'expressions',
 'Division',
 'modulo',
 'Module',
 'Math module',
 'Function',
 'String',
 'List',
 'Tuple',
 'Set',
 'Dictionary',
 'data types summary',
 'Indexing',
 'Type conversions',
 'Binary numbers',
 'Representing text',
 'String formatting',
 'Alternative string formatting',
 'If-else branches',
 'equal values',
 'Comparing',
 'data types',
 'ranges',
 'logical operators',
 'gaps',
 'Membership operators',
 'identity operators',
 'Order of evaluation',
 'Code blocks',
 'indentation',
 'assert statements',
 'User-defined function',
 'Print functions',
 'Dynamic typing',
 'defining functions',
 'mathematical functions',
 'Function stubs',
 'Scope',
 'Namespaces',
 'if __name__ == "__main__"',
 'testing',
 'Help ',
 'docstrings',
 'document functions',
 'common error',
 'Loops',
 'While loops',
 'For loops',
 'range function',
 'While and loops',
 'Nested loops

In [205]:
topics_embedings = [get_768_features(top) for top in topics]

In [206]:
def get_distance(a, b):
    return ((a - b) ** 2).sum()

In [207]:
def cos_sim(a, b):
    """
    Computes the cosine similarity cos_sim(a[i], b[j]) for all i and j.
    :return: Matrix with res[i][j]  = cos_sim(a[i], b[j])
    """
    if not isinstance(a, torch.Tensor):
        a = torch.tensor(a)

    if not isinstance(b, torch.Tensor):
        b = torch.tensor(b)

    if len(a.shape) == 1:
        a = a.unsqueeze(0)

    if len(b.shape) == 1:
        b = b.unsqueeze(0)

    a_norm = torch.nn.functional.normalize(a, p=2, dim=1)
    b_norm = torch.nn.functional.normalize(b, p=2, dim=1)
    return torch.mm(a_norm, b_norm.transpose(0, 1))

In [216]:
area = 0.15
week = 3
target_col = f"week{week}_q{1}_embeding"
df[target_col + "_rel_top"] = [[]] * len(df)
for idx, emb in enumerate(df[target_col]):
    distances = [abs(1 - cos_sim(emb, tope)) for tope in topics_embedings]
    #distances = [get_distance(emb, tope) for tope in topics_embedings]
    mid = min(distances)
    if mid < 0.7:
        relevant = [topics[i] for i in range(len(topics)) if (distances[i] - mid) <= mid * area]
    if not relevant:
        print("Alarm", mid)
    df.iloc[[idx], list(df.columns).index(target_col + "_rel_top")] = ", ".join(relevant)



In [217]:
watch = df[[[i for i in question_one_columns if str(week) in i][0], target_col + "_rel_top"]]

In [218]:
for idx in range(len(watch)):
    obj = list(watch.iloc[idx])
    if len(obj[0]) > 5:
        print(obj[0])
        print("=", obj[1])
        print()

I don't think there was necessarily an example that was challenging, I just struggled learning the differences and when to use a list, tuple, set, and dictionary.
= Dictionary, Binary numbers, Lists, Sorting, Dictionaries, Dictionary methods, Iterating over a dictionary, Dictionary nesting, Writing files

Binary, it just seems difficult to understand and remember how to get specific symbols or numbers.
= Binary numbers

The most challenging lab for me was 3.15. I believe my code is very close to being accurate, but I still do not know what's wrong so Im going to attend office hours this week. I think that I got something about the syntax of adding and indexing strings wrong. 
= Code blocks, Iterating over a dictionary, The 'with' statement

The most challenging concept would be knowing when and why we would use the various types of data storage. More practice with using the new syntax and such helped me get over this.
= Lists, Iterating over a dictionary, Command-line arguments, The 'w

In [212]:
v1 = get_768_features("I don't think there was necessarily an example that was challenging, I just struggled learning the differences and when to use a list, tuple, set, and dictionary.")
v2 = get_768_features("dictionary")
v3 = get_768_features("lists")
v4 = get_768_features("set")
v5 = get_768_features("tuple")

In [213]:
abs(1 - cos_sim(v1, v2)), abs(1 - cos_sim(v1, v3)), abs(1 - cos_sim(v1, v4)), abs(1 - cos_sim(v1, v5))

(tensor([[0.3987]]),
 tensor([[0.4157]]),
 tensor([[0.4577]]),
 tensor([[0.4032]]))

In [215]:
abs(0.3987 - 0.4577) / 0.3987

0.14798093804865814

In [5]:
import numpy as np
order = np.random.shuffled([i for i in range(10)])

AttributeError: module 'numpy.random' has no attribute 'shuffled'

In [4]:
order.shape

AttributeError: 'NoneType' object has no attribute 'shape'